In [ ]:
!pip install -r https://raw.githubusercontent.com/teddylee777/langchain-kr/main/requirements.txt

In [1]:
def get_langchain():
  with open('20250812_LangChain_API.key', 'r', encoding='utf-8') as file:
    return file.readline().rstrip()

In [2]:
def get_openai():
  with open('20250723_OpenAI_API.key', 'r', encoding='utf-8') as file:
    return file.readline().rstrip()

In [3]:
import os

project_name = 'CH03_OutputParsers'

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = project_name
os.environ["LANGCHAIN_API_KEY"] = get_langchain()
os.environ["LANGCHAIN_HUB_API_KEY"] = get_langchain()
os.environ["OPENAI_API_KEY"] = get_openai()

In [5]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith(project_name)

LangSmith 추적을 시작합니다.
[프로젝트명]
CH03_OutputParsers


In [ ]:
from langchain_teddynote import logging

# set_enable=False 로 지정하면 추적을 하지 않습니다.
logging.langsmith(project_name, set_enable=False)

PydanticOuputParser

In [5]:
# 실시간 출력을 위한 import
from langchain_teddynote.messages import stream_response
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field


llm = ChatOpenAI(temperature=0, model_name="gpt-4.1")


In [6]:
email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""


In [7]:
from itertools import chain
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    "다음의 이메일 내용중 중요한 내용을 추출해 주세요.\n\n{email_conversation}"
)

llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

chain = prompt | llm

answer = chain.stream({"email_conversation": email_conversation})

output = stream_response(answer, return_output=True)


이메일의 중요한 내용은 다음과 같습니다:

1. 김철수 상무는 바이크코퍼레이션 소속이며, 이은채 대리에게 이메일을 보냈습니다.
2. 김철수 상무는 "ZENESIS" 자전거에 관심이 있으며, 상세한 브로슈어(기술 사양, 배터리 성능, 디자인 정보 포함)를 요청했습니다.
3. "ZENESIS" 자전거의 유통 협력을 논의하기 위해 1월 15일 화요일 오전 10시에 미팅을 제안했습니다.
4. 미팅 장소는 이은채 대리의 사무실로 제안되었습니다.

In [9]:
class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")


# PydanticOutputParser 생성
parser = PydanticOutputParser(pydantic_object=EmailSummary)


In [10]:
# instruction 을 출력합니다.
print(parser.get_format_instructions())


The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"description": "메일을 보낸 사람", "title": "Person", "type": "string"}, "email": {"description": "메일을 보낸 사람의 이메일 주소", "title": "Email", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문을 요약한 텍스트", "title": "Summary", "type": "string"}, "date": {"description": "메일 본문에 언급된 미팅 날짜와 시간", "title": "Date", "type": "string"}}, "required": ["person", "email", "subject", "summary", "date"]}
```


In [11]:
prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN.

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

# format 에 PydanticOutputParser의 부분 포맷팅(partial) 추가
prompt = prompt.partial(format=parser.get_format_instructions())


In [12]:
# chain 을 생성합니다.
chain = prompt | llm


In [13]:
# chain 을 실행하고 결과를 출력합니다.
response = chain.stream(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

# 결과는 JSON 형태로 출력됩니다.
output = stream_response(response, return_output=True)


```json
{
    "person": "김철수",
    "email": "chulsoo.kim@bikecorporation.me",
    "subject": "\"ZENESIS\" 자전거 유통 협력 및 미팅 일정 제안",
    "summary": "김철수 상무는 바이크코퍼레이션의 자전거 유통 협력을 위해 ZENESIS 모델의 브로슈어를 요청하고, 1월 15일 화요일 오전 10시에 미팅을 제안합니다.",
    "date": "1월 15일 화요일 오전 10시"
}
```

In [14]:
# PydanticOutputParser 를 사용하여 결과를 파싱합니다.
structured_output = parser.parse(output)
print(structured_output)


person='김철수' email='chulsoo.kim@bikecorporation.me' subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안' summary='김철수 상무는 바이크코퍼레이션의 자전거 유통 협력을 위해 ZENESIS 모델의 브로슈어를 요청하고, 1월 15일 화요일 오전 10시에 미팅을 제안합니다.' date='1월 15일 화요일 오전 10시'


In [15]:
# 출력 파서를 추가하여 전체 체인을 재구성합니다.
chain = prompt | llm | parser


In [16]:
# chain 을 실행하고 결과를 출력합니다.
response = chain.invoke(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

# 결과는 EmailSummary 객체 형태로 출력됩니다.
response


EmailSummary(person='김철수', email='chulsoo.kim@bikecorporation.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary='김철수 상무는 바이크코퍼레이션의 자전거 유통 협력을 위해 ZENESIS 모델의 브로슈어를 요청하고, 1월 15일 화요일 오전 10시에 미팅을 제안합니다.', date='1월 15일 화요일 오전 10시')

In [17]:
llm_with_structered = ChatOpenAI(
    temperature=0, model_name="gpt-4.1"
).with_structured_output(EmailSummary)


In [18]:
# invoke() 함수를 호출하여 결과를 출력합니다.
answer = llm_with_structered.invoke(email_conversation)
answer


EmailSummary(person='김철수', email='chulsoo.kim@bikecorporation.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary='김철수 상무(바이크코퍼레이션)가 이은채 대리(테디인터내셔널)에게 신규 자전거 "ZENESIS"의 브로슈어(특히 기술 사양, 배터리 성능, 디자인 정보)를 요청하고, 유통 협력 논의를 위해 1월 15일(화) 오전 10시에 미팅을 제안하는 내용입니다. 미팅 장소는 테디인터내셔널 사무실을 제안했습니다.', date='(날짜 미기재, 미팅 제안일: 1월 15일 오전 10시)')

CommaSeparatedListOutputParser

In [19]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 콤마로 구분된 리스트 출력 파서 초기화
output_parser = CommaSeparatedListOutputParser()

# 출력 형식 지침 가져오기
format_instructions = output_parser.get_format_instructions()
# 프롬프트 템플릿 설정
prompt = PromptTemplate(
    # 주제에 대한 다섯 가지를 나열하라는 템플릿
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],  # 입력 변수로 'subject' 사용
    # 부분 변수로 형식 지침 사용
    partial_variables={"format_instructions": format_instructions},
)

# ChatOpenAI 모델 초기화
model = ChatOpenAI(temperature=0)

# 프롬프트, 모델, 출력 파서를 연결하여 체인 생성
chain = prompt | model | output_parser


In [20]:
# "대한민국 관광명소"에 대한 체인 호출 실행
chain.invoke({"subject": "조지아 관광명소"})


['Stone Mountain Park',
 'Savannah Historic District',
 'Callaway Gardens',
 'Martin Luther King Jr. National Historic Site',
 'Chattahoochee National Forest']

In [22]:
# 스트림을 순회합니다.
for s in chain.stream({"subject": "튀르키예 관광명소"}):
    print(s)  # 스트림의 내용을 출력합니다.


['블루 모스크']
['페티에 궁전']
['카파도키아']
['에페수']
['안탈리아']


StructuredOutputParser

In [23]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI


In [24]:
# 사용자의 질문에 대한 답변
response_schemas = [
    ResponseSchema(name="answer", description="사용자의 질문에 대한 답변"),
    ResponseSchema(
        name="source",
        description="사용자의 질문에 답하기 위해 사용된 `출처`, `웹사이트주소` 이여야 합니다.",
    ),
]
# 응답 스키마를 기반으로 한 구조화된 출력 파서 초기화
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)


In [25]:
# 출력 형식 지시사항을 파싱합니다.
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    # 사용자의 질문에 최대한 답변하도록 템플릿을 설정합니다.
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    # 입력 변수로 'question'을 사용합니다.
    input_variables=["question"],
    # 부분 변수로 'format_instructions'을 사용합니다.
    partial_variables={"format_instructions": format_instructions},
)


In [26]:
model = ChatOpenAI(temperature=0)  # ChatOpenAI 모델 초기화
chain = prompt | model | output_parser  # 프롬프트, 모델, 출력 파서를 연결


In [27]:
# 대한민국의 수도가 무엇인지 질문합니다.
chain.invoke({"question": "호주의 수도는 어디인가요?"})


{'answer': '캔버라(Canberra)가 호주의 수도입니다.',
 'source': 'https://en.wikipedia.org/wiki/Canberra'}

In [28]:
for s in chain.stream({"question": "세종대왕의 업적은 무엇인가요?"}):
    # 스트리밍 출력
    print(s)


{'answer': '세종대왕은 한글을 창제하고 문화를 발전시키는 등 다양한 업적을 남겼습니다.', 'source': 'https://ko.wikipedia.org/wiki/%EC%84%B8%EC%A2%85%EB%8C%80%EC%99%95'}


JsonOutputParser

In [29]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI


In [30]:
# OpenAI 객체를 생성합니다.
model = ChatOpenAI(temperature=0, model_name="gpt-4.1")


In [31]:
# 원하는 데이터 구조를 정의합니다.
class Topic(BaseModel):
    description: str = Field(description="주제에 대한 간결한 설명")
    hashtags: str = Field(description="해시태그 형식의 키워드(2개 이상)")


In [32]:
# 질의 작성
question = "지구 온난화의 심각성 대해 알려주세요."

# 파서를 설정하고 프롬프트 템플릿에 지시사항을 주입합니다.
parser = JsonOutputParser(pydantic_object=Topic)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

prompt = prompt.partial(format_instructions=parser.get_format_instructions())

chain = prompt | model | parser  # 체인을 구성합니다.

chain.invoke({"question": question})  # 체인을 호출하여 쿼리 실행


{'description': '지구 온난화는 온실가스 증가로 인해 지구 평균 기온이 상승하는 현상으로, 해수면 상승, 극심한 기상 이변, 생태계 파괴 등 심각한 문제를 초래합니다.',
 'hashtags': '#지구온난화 #기후변화 #환경위기 #온실가스'}

In [33]:
# 질의 작성
question = "지구 온난화에 대해 알려주세요. 온난화에 대한 설명은 `description`에, 관련 키워드는 `hashtags`에 담아주세요."

# JSON 출력 파서 초기화
parser = JsonOutputParser()

# 프롬프트 템플릿을 설정합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

# 지시사항을 프롬프트에 주입합니다.
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# 프롬프트, 모델, 파서를 연결하는 체인 생성
chain = prompt | model | parser

# 체인을 호출하여 쿼리 실행
response = chain.invoke({"question": question})

# 출력을 확인합니다.
print(response)


{'description': '지구 온난화는 인간 활동으로 인해 대기 중 온실가스 농도가 증가하면서 지구 평균 기온이 점차 상승하는 현상입니다. 이로 인해 해수면 상승, 기후 변화, 극한 기상 현상 등이 발생하며, 생태계와 인류 사회에 다양한 영향을 미칩니다.', 'hashtags': ['#지구온난화', '#기후변화', '#온실가스', '#환경문제', '#지속가능성']}


PandasDataFrameOutputParser

In [34]:
import pprint
from typing import Any, Dict

import pandas as pd
from langchain.output_parsers import PandasDataFrameOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI


In [35]:
# ChatOpenAI 모델 초기화 (gpt-3.5-turbo 모델 사용을 권장합니다)
model = ChatOpenAI(temperature=0, model_name="gpt-4.1")


In [36]:
# 출력 목적으로만 사용됩니다.
def format_parser_output(parser_output: Dict[str, Any]) -> None:
    # 파서 출력의 키들을 순회합니다.
    for key in parser_output.keys():
        # 각 키의 값을 딕셔너리로 변환합니다.
        parser_output[key] = parser_output[key].to_dict()
    # 예쁘게 출력합니다.
    return pprint.PrettyPrinter(width=4, compact=True).pprint(parser_output)


In [37]:
# 원하는 Pandas DataFrame을 정의합니다.
df = pd.read_csv("./titanic.csv")
df.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [38]:
# 파서를 설정하고 프롬프트 템플릿에 지시사항을 주입합니다.
parser = PandasDataFrameOutputParser(dataframe=df)

# 파서의 지시사항을 출력합니다.
print(parser.get_format_instructions())


The output should be formatted as a string as the operation, followed by a colon, followed by the column or row to be queried on, followed by optional array parameters.
1. The column names are limited to the possible columns below.
2. Arrays must either be a comma-separated list of numbers formatted as [1,3,5], or it must be in range of numbers formatted as [0..4].
3. Remember that arrays are optional and not necessarily required.
4. If the column is not in the possible columns or the operation is not a valid Pandas DataFrame operation, return why it is invalid as a sentence starting with either "Invalid column" or "Invalid operation".

As an example, for the formats:
1. String "column:num_legs" is a well-formatted instance which gets the column num_legs, where num_legs is a possible column.
2. String "row:1" is a well-formatted instance which gets row 1.
3. String "column:num_legs[1,2]" is a well-formatted instance which gets the column num_legs for rows 1 and 2, where num_legs is a p

In [39]:
# 열 작업 예시입니다.
df_query = "Age column 을 조회해 주세요."


# 프롬프트 템플릿을 설정합니다.
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],  # 입력 변수 설정
    partial_variables={
        "format_instructions": parser.get_format_instructions()
    },  # 부분 변수 설정
)

# 체인 생성
chain = prompt | model | parser

# 체인 실행
parser_output = chain.invoke({"query": df_query})

# 출력
format_parser_output(parser_output)


{'Age': {0: 22.0,
         1: 38.0,
         2: 26.0,
         3: 35.0,
         4: 35.0,
         5: nan,
         6: 54.0,
         7: 2.0,
         8: 27.0,
         9: 14.0,
         10: 4.0,
         11: 58.0,
         12: 20.0,
         13: 39.0,
         14: 14.0,
         15: 55.0,
         16: 2.0,
         17: nan,
         18: 31.0,
         19: nan}}


In [40]:
# 행 조회 예시입니다.
df_query = "Retrieve the first row."

# 체인 실행
parser_output = chain.invoke({"query": df_query})

# 결과 출력
format_parser_output(parser_output)


{'0': {'Age': 22.0,
       'Cabin': nan,
       'Embarked': 'S',
       'Fare': 7.25,
       'Name': 'Braund, '
               'Mr. '
               'Owen '
               'Harris',
       'Parch': 0,
       'PassengerId': 1,
       'Pclass': 3,
       'Sex': 'male',
       'SibSp': 1,
       'Survived': 0,
       'Ticket': 'A/5 '
                 '21171'}}


In [41]:
# row 0 ~ 4의 평균 나이를 구합니다.
df["Age"].head().mean()


31.2

In [42]:
# 임의의 Pandas DataFrame 작업 예시, 행의 수를 제한합니다.
df_query = "Retrieve the average of the Ages from row 0 to 4."

# 체인 실행
parser_output = chain.invoke({"query": df_query})

# 결과 출력
print(parser_output)


{'mean': 31.2}


In [43]:
# 잘못 형식화된 쿼리의 예시입니다.
df_query = "Calculate average `Fare` rate."

# 체인 실행
parser_output = chain.invoke({"query": df_query})

# 결과 출력
print(parser_output)


{'mean': 22.19937}


DatetimeOutputParser

In [44]:
from langchain.output_parsers import DatetimeOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 날짜 및 시간 출력 파서
output_parser = DatetimeOutputParser()
output_parser.format = "%Y-%m-%d"

# 사용자 질문에 대한 답변 템플릿
template = """Answer the users question:\n\n#Format Instructions: \n{format_instructions}\n\n#Question: \n{question}\n\n#Answer:"""

prompt = PromptTemplate.from_template(
    template,
    partial_variables={
        "format_instructions": output_parser.get_format_instructions()
    },  # 지침을 템플릿에 적용
)

# 프롬프트 내용을 출력
prompt


PromptTemplate(input_variables=['question'], input_types={}, partial_variables={'format_instructions': "Write a datetime string that matches the following pattern: '%Y-%m-%d'.\n\nExamples: 1511-07-13, 1412-12-28, 1514-06-25\n\nReturn ONLY this string, no other words!"}, template='Answer the users question:\n\n#Format Instructions: \n{format_instructions}\n\n#Question: \n{question}\n\n#Answer:')

In [45]:
# Chain 을 생성합니다.
chain = prompt | ChatOpenAI() | output_parser

# 체인을 호출하여 질문에 대한 답변을 받습니다.
output = chain.invoke({"question": "Google 이 창업한 연도는?"})


In [46]:
# 결과를 문자열로 변환
output.strftime("%Y-%m-%d")


'1998-09-04'

EnumOutputParser

In [47]:
from langchain.output_parsers.enum import EnumOutputParser


In [48]:
from enum import Enum


class Colors(Enum):
    RED = "빨간색"
    GREEN = "초록색"
    BLUE = "파란색"


In [49]:
# EnumOutputParser 인스턴스 생성
parser = EnumOutputParser(enum=Colors)


In [50]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 프롬프트 템플릿을 생성합니다.
prompt = PromptTemplate.from_template(
    """다음의 물체는 어떤 색깔인가요?

Object: {object}

Instructions: {instructions}"""
    # 파서에서 지시사항 형식을 가져와 부분적으로 적용합니다.
).partial(instructions=parser.get_format_instructions())
# 프롬프트와 ChatOpenAI, 파서를 연결합니다.
chain = prompt | ChatOpenAI() | parser


In [53]:
response = chain.invoke({"object": "하늘"})  # "하늘" 에 대한 체인 호출 실행
print(response)


Colors.BLUE


OutputFixingParser

In [4]:
from typing import List

from langchain_core.exceptions import OutputParserException
from langchain_core.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

class Actor(BaseModel):
    name: str = Field(description="name of an actor")
    film_names: List[str] = Field(description="list of names of films they starred in")


actor_query = "Generate the filmography for a random actor."

parser = PydanticOutputParser(pydantic_object=Actor)

In [5]:
misformatted = "{'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}"

try:
    parser.parse(misformatted)
except OutputParserException as e:
    print(e)

Invalid json output: {'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 


In [6]:
from langchain.output_parsers import OutputFixingParser

new_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI())

In [7]:
new_parser.parse(misformatted)

Actor(name='Tom Hanks', film_names=['Forrest Gump'])

In [10]:
# OutputFixingParser 를 사용하여 잘못된 형식의 출력을 파싱
actor = new_parser.parse(misformatted)


KeyError: "Input to PromptTemplate is missing variables {'instructions'}.  Expected: ['completion', 'error', 'instructions'] Received: ['completion', 'error']\nNote: if you intended {instructions} to be part of the string and not a variable, please escape it with double curly braces like: '{{instructions}}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT "

In [72]:
fix_prompt = PromptTemplate(
    template="""Given the following JSON output and format instructions, fix the output to match the format instructions:

Format Instructions: {instructions}
Output: {completion}

Fixed JSON Output:""",
    input_variables=["instructions", "completion"]
)

new_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI(), prompt=fix_prompt)

In [73]:
# OutputFixingParser 를 사용하여 잘못된 형식의 출력을 파싱
try:
    # 파싱 시도
    actor = new_parser.parse(misformatted)
    print("파싱 결과:", actor)
    print("배우 이름:", actor.name)
    print("출연 영화:", actor.film_names)
except Exception as e:
    print("오류 발생:", str(e))

오류 발생: "Input to PromptTemplate is missing variables {'instructions'}.  Expected: ['completion', 'instructions'] Received: ['completion', 'error']\nNote: if you intended {instructions} to be part of the string and not a variable, please escape it with double curly braces like: '{{instructions}}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT "


In [59]:
# 파싱된 결과
actor


NameError: name 'actor' is not defined

In [80]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field, ValidationError

# 1. Pydantic 모델 정의
class BlogPost(BaseModel):
  title: str = Field(description="블로그 게시물의 제목")
  summary: str = Field(description="블로그 게시물의 요약")
  keywords: list[str] = Field(description="블로그 게시물의 핵심 키워드 목록")

# 2. PydanticOutputParser 초기화
parser = PydanticOutputParser(pydantic_object=BlogPost)

# 3. 잘못된 형식의 출력 생성 (파싱 실패 유도)
# ```json과 ```를 제거하여 순수한 JSON 형식의 문자열만 남깁니다.
faulty_response_text = """
{
  "title": "잘못된 JSON 예시",
  "summary": "파싱 오류를 유발하기 위한 요약입니다.",
  "keywords": ["키워드1", 키워드2] 
}
"""

print("--- 파싱 오류 테스트 시작 ---")
try:
    # PydanticOutputParser로 파싱 시도 -> ValidationError 발생
    parser.parse(faulty_response_text)
except ValidationError as e:
    print("ValidationError 발생:", e)
    print("\nPydanticOutputParser로 파싱 실패.\n")
except Exception as e:
    # 예상치 못한 다른 에러가 발생할 경우를 대비하여 추가
    print("예상치 못한 에러가 발생했습니다:", e)


# 4. OutputFixingParser를 사용한 오류 처리
from langchain.output_parsers import OutputFixingParser

llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")
fixing_parser = OutputFixingParser.from_llm(parser=parser, llm=llm)

print("--- OutputFixingParser로 복구 시도 ---")
try:
    # OutputFixingParser로 파싱 시도
    # 내부적으로 LLM을 호출하여 잘못된 JSON을 수정합니다.
    repaired_output = fixing_parser.parse(faulty_response_text)

    print("\n성공적으로 복구된 출력:")
    print(f"타입: {type(repaired_output)}")
    print(f"제목: {repaired_output.title}")
    print(f"요약: {repaired_output.summary}")
    print(f"키워드: {repaired_output.keywords}")

    # Pydantic 모델의 인스턴스임을 확인
    if isinstance(repaired_output, BlogPost):
        print("\nOutputFixingParser가 Pydantic 모델 인스턴스를 반환했습니다.")

except Exception as e:
    print("OutputFixingParser로도 복구 실패:", e)

--- 파싱 오류 테스트 시작 ---
예상치 못한 에러가 발생했습니다: Invalid json output: {
  "title": "잘못된 JSON 예시",
  "summary": "파싱 오류를 유발하기 위한 요약입니다.",
  "keywords": ["키워드1", 키워드2] 
}
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 
--- OutputFixingParser로 복구 시도 ---

성공적으로 복구된 출력:
타입: <class '__main__.BlogPost'>
제목: Well-formatted JSON instance
요약: This is a well-formatted JSON instance.
키워드: ['keyword1', 'keyword2']

OutputFixingParser가 Pydantic 모델 인스턴스를 반환했습니다.
